In [1]:
from __future__ import division
import numpy as np
from prng import lcgRandom
from sample import PIKK
from scipy.misc import comb, factorial
from scipy.stats import chisquare, norm 
import scipy.integrate as integrate
import pandas as pd

In [2]:
def getEmpiricalDistr(randomObject, n, k, reps=10**7):
    uniqueSamples = dict()

    for i in range(reps): # use range in python 3, xrange in python 2
        sam = frozenset(PIKK(n, k, randomObject))
        if sam in uniqueSamples:
            uniqueSamples[sam] += 1
        else:
            uniqueSamples[sam] = 1
    return uniqueSamples
    

def getItemCounts(uniqueSamples):
    itemCounts = dict()
    for u,v in uniqueSamples.items():
        for i in u:
            if i in itemCounts:
                itemCounts[i] += v
            else:
                itemCounts[i] = v
    return itemCounts


def printItemFreq(itemCounts, verbose=False):
    for i in itemCounts.keys():
        itemCounts[i] /= reps
        if verbose:
            print(i, itemCounts[i])
    return itemCounts
    

def printMaxProbRatio(itemCounts, verbose=False):
    freq = list(itemCounts.values())
    pr = np.amax(freq)/np.amin(freq)
    if verbose:
        print("Max ratio of selection probs: " + str(pr))
    return pr


def conductChiSquareTest(itemCounts):
    freq = list(itemCounts.values())
    return(chisquare(freq))

In [3]:
def distrNormalRange(w, n):
    '''
    The CDF of the range of n IID standard normals evaluated at w
    '''
    innerInt = lambda x: norm.pdf(x)*(norm.cdf(x+w) - norm.cdf(x))**(n-1)
    tmp = integrate.quad(innerInt, -np.inf, np.inf)
    return n*tmp[0]


def test_distrNormalRange():
    n = 100
    np.random.seed(12345)

    empiricalRangeDistr = np.array([np.ptp(norm.rvs(size=n)) for i in range(100000)])
    for w in np.array(range(6,13))/2:
        emp = np.mean(empiricalRangeDistr <= w)
        theoretical = distrNormalRange(w, n)
        assert np.abs(emp - theoretical) <= 0.005
    return None


def distrMultinomialRange(w, n, k):
    '''
    CDF of the range of multinomial variables, evaluated at w
    n draws, k categories each having probability 1/k
    '''
    cutoff = (w - 1/(2*n))*np.sqrt(k/n)
    return distrNormalRange(cutoff, k)


def test_distrMultinomialRange():
    reps = 10000
    bins = 15
    np.random.seed(12345)

    empiricalRangeDistr = np.ptp(np.random.multinomial(n=reps, pvals=[1/bins]*bins, size=100000), axis=1)
    for w in np.array(range(20))*10:
        emp = np.mean(empiricalRangeDistr <= w)
        theoretical = distrMultinomialRange(w, reps, bins)
        assert np.abs(emp - theoretical) <= 0.05
    return None

# will be silent if there are no errors
test_distrNormalRange()
test_distrMultinomialRange()

In [4]:
# Boilerplate stuff

reps = int(10**7)
n = [13, 30, 90]
k = [4, 10, 20]

maxProb = []
minProb = []
meanProb = []
maxProbRatio = []
nvalues = []
kvalues = []
prng = []
seed = []

# FO = first order selection probabilities
chisqStatistic_FO = []
chisqDF_FO = []
chisqPvalue_FO = []
rangeStat_FO = []
rangePvalue_FO = []

# US = unique sample selection probabilities
chisqStatistic_US = []
chisqDF_US = []
chisqPvalue_US = []
rangeStat_US = []
rangePvalue_US = []

# RANDU

In [5]:
for nn in n:
    for kk in k:
        if kk >= nn:
            continue
        
        lcg = lcgRandom(seed=100) # set seed of RANDU to 100

        uniqueSampleCounts = getEmpiricalDistr(lcg, n=nn, k=kk, reps=reps)
        itemCounts = getItemCounts(uniqueSampleCounts)
        
        # First order
        chisqTestResults = conductChiSquareTest(itemCounts)
        chisqDF_FO = chisqDF_FO + [len(itemCounts)-1]
        chisqStatistic_FO = chisqStatistic_FO + [chisqTestResults[0]]
        chisqPvalue_FO = chisqPvalue_FO + [chisqTestResults[1]]
        
        rangeStatObserved = np.ptp(list(itemCounts.values()))
        rangeStat_FO = rangeStat_FO + [rangeStatObserved]
        rangePvalue_FO = rangePvalue_FO + [1-distrMultinomialRange(rangeStatObserved, reps*kk, nn)]
        
        # Unique samples
        chisqTestResults = conductChiSquareTest(uniqueSampleCounts)
        chisqDF_US = chisqDF_US + [len(uniqueSampleCounts)-1]
        chisqStatistic_US = chisqStatistic_US + [chisqTestResults[0]]
        chisqPvalue_US = chisqPvalue_US + [chisqTestResults[1]]
        
        rangeStatObserved = np.ptp(list(uniqueSampleCounts.values()))
        rangeStat_US = rangeStat_US + [rangeStatObserved]
        rangePvalue_US = rangePvalue_US + [1-distrMultinomialRange(rangeStatObserved, reps, comb(nn, kk))]
        
        # Selection probability summary stats
        itemFreq = printItemFreq(itemCounts)
        maxProb = maxProb + [np.amax(list(itemFreq.values()))]
        minProb = minProb + [np.amin(list(itemFreq.values()))]
        meanProb = meanProb + [np.mean(list(itemFreq.values()))]
        maxProbRatio = maxProbRatio + [printMaxProbRatio(itemFreq)]
        nvalues = nvalues + [nn]
        kvalues = kvalues + [kk]
        prng = prng + ['RANDU']
        seed = seed + [100]

# Super Duper LCG

In [6]:
# Parameters for the Super Duper LCG
A_SD = 0
B_SD = 69069
M_SD = 2**32

In [7]:
seedvalues = [100, 233424280]

for nn in n:
    for kk in k:
        if kk >= nn:
            continue
        for ss in seedvalues:
            sdlcg = lcgRandom(seed=ss, A=A_SD, B=B_SD, M=M_SD)

            uniqueSampleCounts = getEmpiricalDistr(sdlcg, n=nn, k=kk, reps=reps)
            itemCounts = getItemCounts(uniqueSampleCounts)
        
            # First order
            chisqTestResults = conductChiSquareTest(itemCounts)
            chisqDF_FO = chisqDF_FO + [len(itemCounts)-1]
            chisqStatistic_FO = chisqStatistic_FO + [chisqTestResults[0]]
            chisqPvalue_FO = chisqPvalue_FO + [chisqTestResults[1]]
        
            rangeStatObserved = np.ptp(list(itemCounts.values()))
            rangeStat_FO = rangeStat_FO + [rangeStatObserved]
            rangePvalue_FO = rangePvalue_FO + [1-distrMultinomialRange(rangeStatObserved, reps*kk, nn)]
        
            # Unique samples
            chisqTestResults = conductChiSquareTest(uniqueSampleCounts)
            chisqDF_US = chisqDF_US + [len(uniqueSampleCounts)-1]
            chisqStatistic_US = chisqStatistic_US + [chisqTestResults[0]]
            chisqPvalue_US = chisqPvalue_US + [chisqTestResults[1]]
        
            rangeStatObserved = np.ptp(list(uniqueSampleCounts.values()))
            rangeStat_US = rangeStat_US + [rangeStatObserved]
            rangePvalue_US = rangePvalue_US + [1-distrMultinomialRange(rangeStatObserved, reps, comb(nn, kk))]
        
            # Selection probability summary stats
            itemFreq = printItemFreq(itemCounts)
            maxProb = maxProb + [np.amax(list(itemFreq.values()))]
            minProb = minProb + [np.amin(list(itemFreq.values()))]
            meanProb = meanProb + [np.mean(list(itemFreq.values()))]
            maxProbRatio = maxProbRatio + [printMaxProbRatio(itemFreq)]
            nvalues = nvalues + [nn]
            kvalues = kvalues + [kk]
            prng = prng + ['Super-Duper']
            seed = seed + [ss]

# Mersenne Twister

In [8]:
seedvalues = [100, 233424280, 429496729]

for nn in n:
    for kk in k:
        if kk >= nn:
            continue
        for ss in seedvalues:
            mt = np.random
            mt.seed(ss)

            uniqueSampleCounts = getEmpiricalDistr(mt, n=nn, k=kk, reps=reps)
            itemCounts = getItemCounts(uniqueSampleCounts)
        
            # First order
            chisqTestResults = conductChiSquareTest(itemCounts)
            chisqDF_FO = chisqDF_FO + [len(itemCounts)-1]
            chisqStatistic_FO = chisqStatistic_FO + [chisqTestResults[0]]
            chisqPvalue_FO = chisqPvalue_FO + [chisqTestResults[1]]
        
            rangeStatObserved = np.ptp(list(itemCounts.values()))
            rangeStat_FO = rangeStat_FO + [rangeStatObserved]
            rangePvalue_FO = rangePvalue_FO + [1-distrMultinomialRange(rangeStatObserved, reps*kk, nn)]
        
            # Unique samples
            chisqTestResults = conductChiSquareTest(uniqueSampleCounts)
            chisqDF_US = chisqDF_US + [len(uniqueSampleCounts)-1]
            chisqStatistic_US = chisqStatistic_US + [chisqTestResults[0]]
            chisqPvalue_US = chisqPvalue_US + [chisqTestResults[1]]
        
            rangeStatObserved = np.ptp(list(uniqueSampleCounts.values()))
            rangeStat_US = rangeStat_US + [rangeStatObserved]
            rangePvalue_US = rangePvalue_US + [1-distrMultinomialRange(rangeStatObserved, reps, comb(nn, kk))]
        
            # Selection probability summary stats
            itemFreq = printItemFreq(itemCounts)
            maxProb = maxProb + [np.amax(list(itemFreq.values()))]
            minProb = minProb + [np.amin(list(itemFreq.values()))]
            meanProb = meanProb + [np.mean(list(itemFreq.values()))]
            maxProbRatio = maxProbRatio + [printMaxProbRatio(itemFreq)]
            nvalues = nvalues + [nn]
            kvalues = kvalues + [kk]
            prng = prng + ['Mersenne Twister']
            seed = seed + [ss]

# First-order selection probabilities, summary statistics

In [9]:
d = {'Sample size' : kvalues,
     'Pop size' : nvalues,
     'PRNG' : prng,
     'Min Prob' : minProb,
     'Mean Prob' : meanProb,
     'Max Prob' : maxProb,
     'Max Selection Prob Ratio' : maxProbRatio,
     'seed' : seed
    }
resTable = pd.DataFrame(d)
cols = resTable.columns.tolist()
cols.reverse()
cols = [cols[2]] + [cols[1]] + [cols[0]] + cols[3:6] + [cols[7]] + [cols[6]]
resTable[cols].sort_values(['Pop size', 'Sample size', 'PRNG', 'seed'])

,Pop size,Sample size,seed,PRNG,Min Prob,Mean Prob,Max Prob,Max Selection Prob Ratio
24,13,4,100,Mersenne Twister,0.307419,0.307692,0.307860,1.001434
25,13,4,233424280,Mersenne Twister,0.307374,0.307692,0.308173,1.002601
26,13,4,429496729,Mersenne Twister,0.307493,0.307692,0.308019,1.001712
0,13,4,100,RANDU,0.307477,0.307692,0.308064,1.001908
8,13,4,100,Super-Duper,0.307363,0.307692,0.308004,1.002086
9,13,4,233424280,Super-Duper,0.307551,0.307692,0.308008,1.001487
27,13,10,100,Mersenne Twister,0.768851,0.769231,0.769445,1.000772
28,13,10,233424280,Mersenne Twister,0.768923,0.769231,0.769419,1.000646
29,13,10,429496729,Mersenne Twister,0.768897,0.769231,0.769460,1.000732
1,13,10,100,RANDU,0.769090,0.769231,0.769504,1.000539


# First order selection probabilities, chi-squared test and range statistic

We first test whether each item $1, \dots, k$ is selected with equal probability. We do two tests: the usual chi-squared test and another test based on the range of the multinomial values, $max_i n_i - min_i n_i$, where $n_1, \dots, n_k$ are the number of items in each of $k$ cells that have equal probability $1/k$.

Johnson and Young (1960) and Young (1962) provide the following approximation to the distribution of the range

$$P(\max_i n_k - \min_i n_k \leq r) \approx P(W_m \leq (r-(2n)^{-1})(m/n)^{1/2})$$

where $W_m$ denotes the sample range of $m$ independent standard normal random variables. It is a known result (see e.g. Pearson and Hartley p. 43, 1954 or Ruben, 1960) that the distribution function for the range of IID normal samples is given by

$$R(w) = n \int_{-\infty}^{\infty} \phi(x)\left[ \Phi(x+w) - \Phi(x)\right]^{n-1}dx$$

where $\phi$ and $\Phi$ are the standard normal density and cumulative distribution function, respectively.  We leverage these two results to approximate the p-value of the range statistic.

In [10]:
d = {'Sample size' : kvalues,
     'Pop size' : nvalues,
     'PRNG' : prng,
     'Max Selection Prob Ratio' : maxProbRatio,
     'seed' : seed,
     'Chi-squared' : chisqStatistic_FO,
     'Df' : chisqDF_FO,
     'P-value' : chisqPvalue_FO,
     'Range' : rangeStat_FO,
     'Range P-value' : rangePvalue_FO
    }
resTable = pd.DataFrame(d)
cols = ['Pop size', 'Sample size', 'PRNG', 'seed', 'Chi-squared', 'Df', 'P-value', 'Range', 'Range P-value']
resTable[cols].sort_values(['Pop size', 'Sample size', 'PRNG', 'seed'])

,Pop size,Sample size,PRNG,seed,Chi-squared,Df,P-value,Range,Range P-value
24,13,4,Mersenne Twister,100,6.815486,12,0.869562,4409,0.861268
25,13,4,Mersenne Twister,233424280,14.701685,12,0.258157,7994,0.065818
26,13,4,Mersenne Twister,429496729,13.496726,12,0.333993,5264,0.648391
0,13,4,RANDU,100,10.853449,12,0.541518,5867,0.469034
8,13,4,Super-Duper,100,10.596316,12,0.563794,6411,0.319655
9,13,4,Super-Duper,233424280,5.240339,12,0.949466,4574,0.827498
27,13,10,Mersenne Twister,100,3.604493,12,0.989563,5938,0.953381
28,13,10,Mersenne Twister,233424280,3.327856,12,0.992753,4965,0.989036
29,13,10,Mersenne Twister,429496729,5.064338,12,0.955795,5631,0.968946
1,13,10,RANDU,100,2.426295,12,0.998416,4143,0.997911


# Selection probabilities for unique samples, chi-squared test + range test

In [11]:
d = {'Sample size' : kvalues,
     'Pop size' : nvalues,
     'PRNG' : prng,
     'Max Selection Prob Ratio' : maxProbRatio,
     'seed' : seed,
     'Chi-squared' : chisqStatistic_US,
     'Df' : chisqDF_US,
     'P-value' : chisqPvalue_US,
     'Range' : rangeStat_US,
     'Range P-value' : rangePvalue_US
    }
resTable = pd.DataFrame(d)
cols = ['Pop size', 'Sample size', 'PRNG', 'seed', 'Chi-squared', 'Df', 'P-value', 'Range', 'Range P-value']
resTable[cols].sort_values(['Pop size', 'Sample size', 'PRNG', 'seed'])

,Pop size,Sample size,PRNG,seed,Chi-squared,Df,P-value,Range,Range P-value
24,13,4,Mersenne Twister,100,6.999703e+02,714,6.391000e-01,648,9.639159e-01
25,13,4,Mersenne Twister,233424280,7.150027e+02,714,4.823872e-01,732,5.424759e-01
26,13,4,Mersenne Twister,429496729,7.086343e+02,714,5.496169e-01,886,1.916390e-02
0,13,4,RANDU,100,9.597907e+02,714,1.856374e-09,846,5.683611e-02
8,13,4,Super-Duper,100,6.668423e+02,714,8.959426e-01,740,4.877451e-01
9,13,4,Super-Duper,233424280,6.731836e+02,714,8.607474e-01,669,9.077825e-01
27,13,10,Mersenne Twister,100,2.740896e+02,285,6.681940e-01,1084,4.139850e-01
28,13,10,Mersenne Twister,233424280,2.509323e+02,285,9.278418e-01,1079,4.328859e-01
29,13,10,Mersenne Twister,429496729,2.887823e+02,285,4.263701e-01,1281,3.184493e-02
1,13,10,RANDU,100,5.618441e+02,285,2.624828e-20,1634,2.494829e-05


# SHA-256 PRNG

In [12]:
# Boilerplate stuff
from sha256prng import SHA256
reps = int(10**7)
n = [13, 30]
k = [4, 10]

maxProb = []
minProb = []
meanProb = []
maxProbRatio = []
nvalues = []
kvalues = []
prng = []
seed = []

# FO = first order selection probabilities
chisqStatistic_FO = []
chisqDF_FO = []
chisqPvalue_FO = []
rangeStat_FO = []
rangePvalue_FO = []

# US = unique sample selection probabilities
chisqStatistic_US = []
chisqDF_US = []
chisqPvalue_US = []
rangeStat_US = []
rangePvalue_US = []
seedvalues = [100, 233424280, 429496729]

In [13]:
for nn in n:
    for kk in k:
        if kk >= nn:
            continue
        for ss in seedvalues:
            prng = SHA256(ss)

            uniqueSampleCounts = getEmpiricalDistr(prng, n=nn, k=kk, reps=reps)
            itemCounts = getItemCounts(uniqueSampleCounts)
        
            # First order
            chisqTestResults = conductChiSquareTest(itemCounts)
            chisqDF_FO = chisqDF_FO + [len(itemCounts)-1]
            chisqStatistic_FO = chisqStatistic_FO + [chisqTestResults[0]]
            chisqPvalue_FO = chisqPvalue_FO + [chisqTestResults[1]]
        
            rangeStatObserved = np.ptp(list(itemCounts.values()))
            rangeStat_FO = rangeStat_FO + [rangeStatObserved]
            rangePvalue_FO = rangePvalue_FO + [1-distrMultinomialRange(rangeStatObserved, reps*kk, nn)]
        
            # Unique samples
            chisqTestResults = conductChiSquareTest(uniqueSampleCounts)
            chisqDF_US = chisqDF_US + [len(uniqueSampleCounts)-1]
            chisqStatistic_US = chisqStatistic_US + [chisqTestResults[0]]
            chisqPvalue_US = chisqPvalue_US + [chisqTestResults[1]]
        
            rangeStatObserved = np.ptp(list(uniqueSampleCounts.values()))
            rangeStat_US = rangeStat_US + [rangeStatObserved]
            rangePvalue_US = rangePvalue_US + [1-distrMultinomialRange(rangeStatObserved, reps, comb(nn, kk))]
        
            # Selection probability summary stats
            itemFreq = printItemFreq(itemCounts)
            maxProb = maxProb + [np.amax(list(itemFreq.values()))]
            minProb = minProb + [np.amin(list(itemFreq.values()))]
            meanProb = meanProb + [np.mean(list(itemFreq.values()))]
            maxProbRatio = maxProbRatio + [printMaxProbRatio(itemFreq)]
            nvalues = nvalues + [nn]
            kvalues = kvalues + [kk]
            seed = seed + [ss]

# First order selection probabilities, chi-squared test and range statistic

In [14]:
d = {'Sample size' : kvalues,
     'Pop size' : nvalues,
     'Max Selection Prob Ratio' : maxProbRatio,
     'seed' : seed,
     'Chi-squared' : chisqStatistic_FO,
     'Df' : chisqDF_FO,
     'P-value' : chisqPvalue_FO,
     'Range' : rangeStat_FO,
     'Range P-value' : rangePvalue_FO
    }
resTable = pd.DataFrame(d)
cols = ['Pop size', 'Sample size', 'seed', 'Chi-squared', 'Df', 'P-value', 'Range', 'Range P-value']
resTable[cols].sort_values(['Pop size', 'Sample size', 'seed'])

,Pop size,Sample size,seed,Chi-squared,Df,P-value,Range,Range P-value
0,13,4,100,7.928909,12,0.790661,4584,0.825319
1,13,4,233424280,9.141152,12,0.690829,5495,0.579990
2,13,4,429496729,7.957963,12,0.788407,5572,0.556905
3,13,10,100,3.114602,12,0.994679,4040,0.998362
4,13,10,233424280,3.647559,12,0.988990,5427,0.976864
5,13,10,429496729,5.543758,12,0.937315,6688,0.893119
6,30,4,100,15.910151,29,0.976535,3641,0.923548
7,30,4,233424280,19.062388,29,0.919678,4140,0.756459
8,30,4,429496729,15.090435,29,0.984297,3142,0.987619
9,30,10,100,11.073729,29,0.998922,4936,0.988698


# Selection probabilities for unique samples, chi-squared test + range test

In [15]:
d = {'Sample size' : kvalues,
     'Pop size' : nvalues,
     'Max Selection Prob Ratio' : maxProbRatio,
     'seed' : seed,
     'Chi-squared' : chisqStatistic_US,
     'Df' : chisqDF_US,
     'P-value' : chisqPvalue_US,
     'Range' : rangeStat_US,
     'Range P-value' : rangePvalue_US
    }
resTable = pd.DataFrame(d)
cols = ['Pop size', 'Sample size', 'seed', 'Chi-squared', 'Df', 'P-value', 'Range', 'Range P-value']
resTable[cols].sort_values(['Pop size', 'Sample size', 'seed'])

,Pop size,Sample size,seed,Chi-squared,Df,P-value,Range,Range P-value
0,13,4,100,6.629075e+02,714,0.914339,683,0.849089
1,13,4,233424280,6.953674e+02,714,0.684296,747,0.441119
2,13,4,429496729,6.872902e+02,714,0.757452,700,0.756063
3,13,10,100,2.689502e+02,285,0.744487,1018,0.675567
4,13,10,233424280,3.062050e+02,285,0.185442,1088,0.399118
5,13,10,429496729,3.207855e+02,285,0.071048,1178,0.145520
6,30,4,100,2.746469e+04,27404,0.396706,159,0.331846
7,30,4,233424280,2.739887e+04,27404,0.507612,174,0.024184
8,30,4,429496729,2.773797e+04,27404,0.077276,152,0.691028
9,30,10,100,1.335685e+06,8507168,1.000000,6,1.000000
